# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
from pprint import pprint

# Import API key
from api_keys import g_key
gmaps.configure(api_key=g_key)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
weatherCsvPath = "output_data/city_weather_data.csv"
weather_df = pd.read_csv(weatherCsvPath)
weather_df

,City,Latitude,Longitude,Temperature,Humidity,Cloudiness,Wind Speed,Country,Date
0,albany,42.60,-73.97,28.78,86,36,3.67,US,1604123172
1,mvuma,-19.28,30.53,73.35,40,0,13.38,ZW,1604123553
2,mar del plata,-38.00,-57.56,48.52,93,0,5.82,AR,1604123259
3,barrow,71.29,-156.79,3.20,84,20,6.93,US,1604123102
4,east london,-33.02,27.91,59.00,87,20,6.93,ZA,1604123179
...,...,...,...,...,...,...,...,...,...
555,panjab,34.39,67.02,46.31,21,0,1.95,AF,1604123250
556,akyab,20.15,92.90,84.40,69,86,6.40,MM,1604123789
557,champerico,14.30,-91.92,64.02,96,28,7.63,GT,1604123264
558,la dorada,5.45,-74.66,72.57,87,41,0.96,CO,1604123789


In [3]:
#Turning lat and lon into a list of tuples
latLongDF = weather_df.loc[:, ["Latitude", "Longitude"]]
coordinatesDF = latLongDF.to_records(index=False)
coordinates = list(coordinatesDF)

### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [4]:
figure_layout = {
    'width': '800px',
    'height': '450px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}
fig = gmaps.figure(layout=figure_layout)
heat_layer = gmaps.heatmap_layer(coordinates, weights=weather_df["Humidity"], 
                                 dissipating=False, max_intensity=100,
                                 point_radius=1)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(border='1px solid black', height='450px', margin='0 auto 0 auto', padding='1px', wi…

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
idealWeather_df = weather_df.loc[(weather_df["Temperature"] > 60) & (weather_df["Temperature"] < 75) & (weather_df["Wind Speed"] < 10) & (weather_df["Cloudiness"] < 20), :]
hotel_df = idealWeather_df.head(16).reset_index(drop=True)
hotel_df

,City,Latitude,Longitude,Temperature,Humidity,Cloudiness,Wind Speed,Country,Date
0,marzuq,14.40,46.47,73.04,24,0,3.42,YE,1604123325
1,chaman,30.92,66.45,65.62,21,0,3.94,PK,1604123678
2,gushi,34.64,109.58,67.14,25,4,4.21,CN,1604123690
3,mattru,7.62,-11.83,73.24,96,9,1.32,SL,1604123691
4,kapaa,22.08,-159.32,73.36,73,1,4.70,US,1604123224
5,tabas,33.60,56.92,64.40,22,0,2.24,IR,1604123700
6,rafsanjan,30.41,55.99,64.94,23,0,0.94,IR,1604123701
7,bardiyah,31.76,25.09,68.59,69,19,9.19,LY,1604123339
8,arlit,18.74,7.39,71.64,22,0,9.04,NE,1604123232
9,puerto del rosario,28.50,-13.86,69.80,73,0,6.93,ES,1604123726


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
hotel_df["Hotel Name"] = ""
hotel_df

,City,Latitude,Longitude,Temperature,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,marzuq,14.40,46.47,73.04,24,0,3.42,YE,1604123325,
1,chaman,30.92,66.45,65.62,21,0,3.94,PK,1604123678,
2,gushi,34.64,109.58,67.14,25,4,4.21,CN,1604123690,
3,mattru,7.62,-11.83,73.24,96,9,1.32,SL,1604123691,
4,kapaa,22.08,-159.32,73.36,73,1,4.70,US,1604123224,
5,tabas,33.60,56.92,64.40,22,0,2.24,IR,1604123700,
6,rafsanjan,30.41,55.99,64.94,23,0,0.94,IR,1604123701,
7,bardiyah,31.76,25.09,68.59,69,19,9.19,LY,1604123339,
8,arlit,18.74,7.39,71.64,22,0,9.04,NE,1604123232,
9,puerto del rosario,28.50,-13.86,69.80,73,0,6.93,ES,1604123726,


In [8]:
# params dictionary to update each iteration
params = {
    "radius": 50000,
    "types": "lodging",
    "keyword": "hotel",
    "key": g_key
}

for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat = row["Latitude"]
    lng = row["Longitude"]

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"

    # Use the search term: "International Airport" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request and print url
    name_address = requests.get(base_url, params=params).json()

    # Since some data may be missing we incorporate a try-except to skip any that are missing a data point.
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["results"][0]["name"]
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")

In [9]:
trimmedHotel_df = hotel_df.iloc[range(0,10),:]
trimmedHotel_df

,City,Latitude,Longitude,Temperature,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,marzuq,14.40,46.47,73.04,24,0,3.42,YE,1604123325,Shabwah Hotel
1,chaman,30.92,66.45,65.62,21,0,3.94,PK,1604123678,Sakhy Hotal Wah Mosafir Khana
2,gushi,34.64,109.58,67.14,25,4,4.21,CN,1604123690,Time Tunnel Designer Hotel
3,mattru,7.62,-11.83,73.24,96,9,1.32,SL,1604123691,Dohas Hotel And Restaurant
4,kapaa,22.08,-159.32,73.36,73,1,4.70,US,1604123224,Kauai Beach Resort
5,tabas,33.60,56.92,64.40,22,0,2.24,IR,1604123700,Amoonorooz Traditional Guest House
6,rafsanjan,30.41,55.99,64.94,23,0,0.94,IR,1604123701,Hilan Hotel
7,bardiyah,31.76,25.09,68.59,69,19,9.19,LY,1604123339,Hotel Burdy
8,arlit,18.74,7.39,71.64,22,0,9.04,NE,1604123232,Hôtel Telwa Bungalow
9,puerto del rosario,28.50,-13.86,69.80,73,0,6.93,ES,1604123726,Hotel Riu Oliva Beach Resort


In [10]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in trimmedHotel_df.iterrows()]
locations = trimmedHotel_df[["Latitude", "Longitude"]]

In [11]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations)
fig.add_layer(markers)
fig
# Display figure


Figure(layout=FigureLayout(border='1px solid black', height='450px', margin='0 auto 0 auto', padding='1px', wi…